# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [41]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [42]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [43]:
spaceship.shape

(8693, 14)

In [44]:
spaceship.isnull ().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [45]:
spaceship = spaceship.dropna ()

In [46]:
spaceship["Cabin"] = spaceship["Cabin"].str[0]

In [47]:
spaceship = spaceship.drop(columns=["PassengerId", "Name"], axis=1)

In [48]:
categorical_columns = spaceship.select_dtypes(exclude=["int64", "float64"]).columns

spaceship_df = pd.get_dummies(spaceship, columns=categorical_columns, drop_first=True)
spaceship_df

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True,Transported_True
0,39.0,0.0,0.0,0.0,0.0,0.0,True,False,False,True,False,False,False,False,False,False,False,True,False,False
1,24.0,109.0,9.0,25.0,549.0,44.0,False,False,False,False,False,False,False,True,False,False,False,True,False,True
2,58.0,43.0,3576.0,0.0,6715.0,49.0,True,False,False,False,False,False,False,False,False,False,False,True,True,False
3,33.0,0.0,1283.0,371.0,3329.0,193.0,True,False,False,False,False,False,False,False,False,False,False,True,False,False
4,16.0,303.0,70.0,151.0,565.0,2.0,False,False,False,False,False,False,False,True,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,True,False,False,False,False,False,False,False,False,False,False,False,True,False
8689,18.0,0.0,0.0,0.0,0.0,0.0,False,False,True,False,False,False,False,False,True,False,True,False,False,False
8690,26.0,0.0,0.0,1872.0,1.0,0.0,False,False,False,False,False,False,False,False,True,False,False,True,False,True
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,True,False,False,False,False,False,True,False,False,False,False,False,False,False


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [49]:
#your code here
features = spaceship_df.drop (columns = ["Transported_True"])

target = spaceship_df ["Transported_True"]

In [50]:

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

In [51]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [52]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [53]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns, index= X_train.index)
X_train_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
3432,0.405063,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7312,0.050633,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2042,0.379747,0.00000,0.007916,0.000000,0.051276,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4999,0.215190,0.00131,0.000000,0.046111,0.016378,0.000049,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5755,0.329114,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [54]:
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns, index= X_test.index)
X_test_norm.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
2453,0.632911,0.0,0.000000,0.0,0.00000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1334,0.227848,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
8272,0.189873,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5090,0.658228,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4357,0.784810,0.0,0.054775,0.0,0.07774,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [55]:
#your code here
#Using Bagging and Pasting

bagging_class = BaggingClassifier(DecisionTreeClassifier(max_depth=20),
                               n_estimators=100, # number of models to use
                               max_samples = 1000)



In [56]:
bagging_class.fit(X_train_norm, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=20),
                  max_samples=1000, n_estimators=100)

- Evaluate your model

In [58]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

pred = bagging_class.predict(X_test_norm)


print(f"Accuracy: {accuracy_score(y_test, pred):.2f}")
print(f"Precision: {precision_score(y_test, pred):.2f}")  
print(f"Recall: {recall_score(y_test, pred):.2f}")
print(f"F1 Score: {f1_score(y_test, pred):.2f}")

# Confusion Matrix
print(f"Confusion Matrix:\n{confusion_matrix(y_test, pred)}")

Accuracy: 0.78
Precision: 0.78
Recall: 0.79
F1 Score: 0.78
Confusion Matrix:
[[512 149]
 [139 522]]


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [59]:
#your code here
grid = {"max_leaf_nodes": [250, 500, 1000],
        "max_depth":[10,30,50]}

In [60]:
dt = DecisionTreeClassifier()

- Run Grid Search

In [61]:
model = GridSearchCV(estimator = dt, param_grid = grid, cv=5, verbose=10) 

In [62]:
model.fit(X_train_norm, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 1/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.792 total time=   0.0s
[CV 2/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 2/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.771 total time=   0.0s
[CV 3/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 3/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.769 total time=   0.0s
[CV 4/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 4/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.751 total time=   0.0s
[CV 5/5; 1/9] START max_depth=10, max_leaf_nodes=250............................
[CV 5/5; 1/9] END max_depth=10, max_leaf_nodes=250;, score=0.780 total time=   0.0s
[CV 1/5; 2/9] START max_depth=10, max_leaf_nodes=500............................
[CV 1/5; 2/9] END max_depth=10, ma

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [10, 30, 50],
                         'max_leaf_nodes': [250, 500, 1000]},
             verbose=10)

In [63]:
model.best_params_

{'max_depth': 30, 'max_leaf_nodes': 250}

In [64]:
best_model = model.best_estimator_

- Evaluate your model

In [65]:
pred = best_model.predict(X_test_norm)


print(f"Accuracy: {accuracy_score(y_test, pred):.2f}")
print(f"Precision: {precision_score(y_test, pred):.2f}")  
print(f"Recall: {recall_score(y_test, pred):.2f}")
print(f"F1 Score: {f1_score(y_test, pred):.2f}")

# Confusion Matrix
print(f"Confusion Matrix:\n{confusion_matrix(y_test, pred)}")

Accuracy: 0.77
Precision: 0.76
Recall: 0.80
F1 Score: 0.78
Confusion Matrix:
[[492 169]
 [129 532]]
